<a href="https://colab.research.google.com/github/gbessardon/Land_Cover_comparison/blob/main/ESA_CCI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Running environment options

In [18]:
colab=1 # 1 running on google collab
drive=1 # 1 need to mount google drive

## LUCAS file location

In [19]:
fnshp='/content/drive/MyDrive/LUCAS2018/LUCAS_2018_Copernicus_polygons.shp'
fncsv='/content/drive/MyDrive/LUCAS2018/LUCAS_2018_Copernicus_attributes.csv'

## ESA-CCI file location

In [20]:
fnESA='/content/drive/MyDrive/ESA-CCI/C3S-LC-L4-LCCS-Map-300m-P1Y-2018-v2.1.1.nc'
ESAfolder=fnESA.replace('/C3S-LC-L4-LCCS-Map-300m-P1Y-2018-v2.1.1.nc','')

# Run on collab and drive 
set colab=1 and drive=1  in running enviroment options

## MOUNT google drive

In [21]:
if drive==1:
  from google.colab import drive
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## install geopandas and pyrpoj for collab

In [22]:
if colab==1:
  !pip install geopandas
  !pip install pyproj
  !pip install rioxarray

# import libraires

In [23]:
import os
from osgeo import gdal
from shapely import geometry
import geopandas as gpd
import numpy as np
import shutil
import xarray as xr
import rioxarray as rio

# Create functions

## create directory to store the segment before zipping

In [24]:
def create_sgement_dir(path=os.getcwd(),ending='LUCAS_Segment'):
    outputdir=os.path.join(path,ending)
    if not os.path.isdir(outputdir):
      os.mkdir(outputdir)
    return outputdir

## CUT ESA-CCI segments

In [25]:
def cutESACCI(ncfile,bounds,outfn):
  #Crop to a sligthly larger area
  LONMIN=bounds[0]-0.1
  LATMIN=bounds[1]-0.1
  LONMAX=bounds[2]+0.1
  LATMAX=bounds[3]+0.1
  LAT=ncfile['lat'][(ncfile['lat']<LATMAX)&(ncfile['lat']>LATMIN)]
  LON=ncfile['lon'][(ncfile['lon']<LONMAX)&(ncfile['lon']>LONMIN)]
  var=ncfile['lccs_class'][0,(ncfile['lat']<LATMAX)&(ncfile['lat']>LATMIN),
                          (ncfile['lon']<LONMAX)&(ncfile['lon']>LONMIN)]
  var.rio.write_crs("epsg:4326", inplace=True)
  var.rio.to_raster('temp.tif')
  # Recrop to the actual area and increase the resolution  
  Pixelsize=[l for l in gdal.Info('temp.tif',format='text').split('\n') if l.startswith('Pixel')][0]
  xres=float(Pixelsize.replace('(','').replace(')','').split('=')[1].split(',')[0])
  yres=float(Pixelsize.replace('(','').replace(')','').split('=')[1].split(',')[1])
  opt=gdal.WarpOptions(dstSRS="EPSG:4326",outputBounds=bounds,xRes=xres/30,yRes=yres/30)
  gdal.Warp(outfn,'temp.tif',options=opt)
  return

# Main

## Open LUCAS shapefile in a geodatabase

In [26]:
LUCASgdf = gpd.read_file(fnshp)

## Create directory to store the segment before zipping

In [27]:
outputdir=create_sgement_dir(path=os.getcwd(),ending='LUCAS_Segment_ESA')

## Open ESACCI

In [28]:
ncfile = xr.open_dataset(fnESA)

## Create ESACCI segments corresponding to the LUCAS segments

In [ ]:
for i,d in LUCASgdf.iterrows():
      lucasp=LUCASgdf['geometry'].loc[i]
      bounds=lucasp.bounds
      lucasid=LUCASgdf['POINT_ID'].loc[i]
      outfile=os.path.join(outputdir,str(lucasid)+'.tif')
      cutESACCI(ncfile,bounds,outfile)

## Save in a zip archive in google drive (prevents I/O errors)

In [ ]:
shutil.make_archive(os.path.join(ESAfolder,'LUCAS_Segment'), 'zip', outputdir)